# Main Libraries for the Project

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Reading the  Train Dataset and Checking if has missing Values

In [ ]:
trainData = pd.read_csv("/Users/jeremyperez/Jupyter/NSL-KDD/KDDTrain+.csv", header = None) 
#Run a Missing Value Ratio test to determine if any feature is missing values.
#If all ratios = 0.0, then data is not missing any values for any features.

#More info about Missing value ratio at 
#https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/

trainData.isnull().sum()/len(trainData)*100

# Reading the  Test Dataset and Checking if has missing Values

In [ ]:
testData = pd.read_csv("/Users/jeremyperez/Jupyter/NSL-KDD/KDDTest+.csv", header = None)
#Run a Missing Value Ratio test to determine if any feature is missing values.
#If all ratios = 0.0, then data is not missing any values for any features.
testData.isnull().sum()/len(testData)*100

# Getting the Dependent and independent Variables

In [ ]:
X = trainData.iloc[:,:-1].values # Get all the rows and all the clums except all the colums - 1
Y = trainData.iloc[:,42].values# Get all the rows and the colum number 42
A = testData.iloc[:,:-1].values # Get all the rows and all the clums except all the colums - 1
Z = testData.iloc[:,42].values# Get all the rows and the colum number 42
attacks = trainData.iloc[:,42].values #Attacks with no one hot encoding

X = pd.DataFrame(X)
Y = pd.DataFrame(Y)
A = pd.DataFrame(A)
Z = pd.DataFrame(Z)

# Encoding Categorical Data for Train Set

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
#We use One hot encoding to pervent the machine learning to atribute the categorical data in order. 
#What one hot encoding(ColumnTransformer) does is, it takes a column which has categorical data, 
#which has been label encoded, and then splits the column into multiple columns.
#The numbers are replaced by 1s and 0s, depending on which column has what value
#We don't need to do a label encoded step because ColumnTransformer do one hot encode and label encode!

#Encoding the Independient Variable
transformX = ColumnTransformer([("Servers", OneHotEncoder(categories = "auto"), [1,2,3])], remainder="passthrough")
X = transformX.fit_transform(X)
#Encoding the Dependent Variable
transformY= ColumnTransformer([("Attacks", OneHotEncoder(categories = "auto"), [0])], remainder="passthrough")
Y = transformY.fit_transform(Y)

# Encoding Categorical Data for Test Set

In [ ]:
#Encoding the Independient Variable
transformA = ColumnTransformer([("Servers", OneHotEncoder(categories = "auto"), [1,2,3])], remainder="passthrough")
A = transformA.fit_transform(A)
    
#Encoding the Dependent Variable
transformZ = ColumnTransformer([("Attacks", OneHotEncoder(categories = "auto"), [0])], remainder="passthrough")
Z = transformZ.fit_transform(Z)

# Scaling the data with Standard Scaling

In [ ]:
#Because we are using numerical-value-only clustering techniques to analyze the NSL-KDD dataset,
#we need to normalize the values in the dataset, as Ibrahim., et. al. describe (page 112).
#We complete the normalization process below:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

trainScaler = StandardScaler()
trainX = trainScaler.fit_transform(X)

testScaler = StandardScaler()
testA = testScaler.fit_transform(A)


trainData = trainX #np.array(trainX)
trainLabel = Y #np.array(Y)

testData =  testA #np.array(testA)
testLabel =  Z #np.array(Z)

#model = LogisticRegression(solver = 'lbfgs')
#model.fit(trainData,trainLabel)

# Elbow Method 

In [ ]:
#Elbow method to find the best number of culster
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++',max_iter = 300,n_init = 10,random_state = 0)
    kmeans.fit(trainData)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
#5 clusters 

# KMeans

In [ ]:
#Applying K-mea(n_clusters = 5)
KMEANS = KMeans(n_clusters = 4, init = 'k-means++',max_iter = 300,n_init = 10,random_state = 0)
kmeans = KMEANS.fit(trainData)
kmeans.labels_

# Analyzing Results by Crosstab

In [ ]:
pd.crosstab(attacks,kmeans.labels_)

# Analyzing Results by Graph

In [ ]:
#Visual representation of the clusters
#plt.scatter(X[y_kmeans ==0,0],X[y_kmeans == 0,1], s = 21, c = 'red', label = 'cluster1')
#plt.scatter(X[y_kmeans ==1,0],X[y_kmeans == 1,1], s = 21, c = 'yellow', label = 'cluster2')
#plt.scatter(X[y_kmeans ==2,0],X[y_kmeans == 2,1], s = 21, c = 'cyan', label = 'cluster3')
#plt.scatter(X[y_kmeans ==3,0],X[y_kmeans == 3,1], s = 21, c = 'orange', label = 'cluster4')
#plt.scatter(X[y_kmeans ==4,0],X[y_kmeans == 4,1], s = 21, c = 'black', label = 'cluster5')
#plt.scatter(kmeans.cluster_centers_[:, 0],kmeans.cluster_centers_[:, 1],s = 300, c = 'purple', label = 'Centroids')
#plt.title('Clusters of Attacks')
#plt.xlabel('Numbers of Attacks')
#plt.ylabel('Types of Attacks')
#plt.legend()
#plt.show()

# DBSCAN 

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score
#OPTICS
# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(trainData)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

pd.crosstab(attacks,labels)


# F-Score implementation

In [ ]:
from sklearn.metrics import f1_score

fscore = f1_score(X,Y).fit(X)
print(fscore)